In [3]:
import numpy
import pandas
print("NumPy:", numpy.__version__)
print("Pandas:", pandas.__version__)

NumPy: 1.26.4
Pandas: 2.3.1


In [2]:
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Step 1: Load dataset
df = pd.read_csv(r"C:\Users\rcish\Downloads\loan_data.csv\Training Dataset.csv")  # Make sure the file exists in your working directory
df = df.dropna().astype(str)
documents = df.apply(lambda row: " | ".join(row), axis=1).tolist()

# Step 2: Embed documents
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(documents, convert_to_numpy=True)

# Step 3: Create FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

# Step 4: Load generation model
generator = pipeline("text2text-generation", model="google/flan-t5-base")

# Step 5: Function to answer questions
def answer_question(question, k=3):
    query_embedding = embedder.encode([question])
    distances, indices = index.search(query_embedding, k)
    context = "\n".join([documents[i] for i in indices[0]])

    prompt = f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    output = generator(prompt, max_length=200, do_sample=True)[0]['generated_text']
    return output

# Step 6: Chat loop
print("🤖 RAG Chatbot Ready. Ask about loan approvals (type 'exit' to quit)\n")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    print("Bot:", answer_question(user_input))

Device set to use cpu


🤖 RAG Chatbot Ready. Ask about loan approvals (type 'exit' to quit)



You:  What factors affect loan approval?


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Bot: Male | Female | 3+ | Not Graduate | No | 3522 | 0.0 | 81.0 | 180.0 | 1.0 | Rural | N | LP001207 | Male | Yes | 0 | Not Graduate | Yes | 2609 | 3449.0 | 165.0 | 180.0 | 0.0 | Rural | N


You:  history


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Bot: LP002772 is a list of women's and men's records from the 18th and 19th centuries. LP001917 is a list of women's and men's records from the 18th and 19th centuries. LP001917 is a list of women's and men's records from the 18th and 19th centuries. LP001917 is a list of women's and men's records from the 18th and 19th centuries. LP001917 is a list of women's and men's records from the 18th and 19th centuries. LP001917 is a list of women's and men's records from the 18th and 19th centuries. LP001917 is a list of women's and men's records from the 18th and 19th centuries. LP001917 is a list of women's and men's records from the 18th and 19th centuries. LP001917 is a list of women's and men's records from the 18th and 19th centuries. LP001917 is a list of women's


You:  exit
